# Importar librerias

# Cargar datos

In [2]:
import pandas as pd

# Cargar datos desde la URL
url = "https://raw.githubusercontent.com/ignaciomsarmiento/RecomSystemsLectures/main/L07_sentimientos/data/Amazon.csv"
data = pd.read_csv(url)

# Verificar las columnas y una muestra de los datos
print(data.columns)
print(data.head())

# Asegurarse de que las columnas relevantes están disponibles
required_columns = ['reviews.text_esp', 'reviews.rating']
missing_columns = [col for col in required_columns if col not in data.columns]
if missing_columns:
    raise ValueError(f"Las siguientes columnas necesarias están ausentes en los datos: {missing_columns}")

# Si necesitas procesar datos adicionales
data = data[['reviews.text_esp', 'reviews.rating']].copy()

# Agregar una columna de "sentimiento" basada en el valor de 'reviews.rating' (opcional)
# Por ejemplo, suponiendo: >= 4.0 es positivo, < 4.0 es negativo
data['sentimiento'] = data['reviews.rating'].apply(lambda x: 'positivo' if x >= 4.0 else 'negativo')

# Mostrar una muestra procesada
print(data.head())


Index(['Unnamed: 0', 'id', 'reviews.text', 'reviews.rating',
       'reviews.text_esp'],
      dtype='object')
   Unnamed: 0                    id  \
0           0  AVqVGWQDv8e3D1O-ldFr   
1           1  AVqkIhwDv8e3D1O-lebb   
2           2  AVphgVaX1cnluZ0-DR74   
3           3  AVphgVaX1cnluZ0-DR74   
4           4  AVqVGWLKnnc1JgDc3jF1   

                                        reviews.text  reviews.rating  \
0  This is a very nice tablet for my GF who has n...             5.0   
1  Love this tablet. Easy to use. And price was r...             5.0   
2  Affordable price awesome quality I love my Ama...             5.0   
3  I bought this after speaking with a sales rep ...             5.0   
4  Bought this tablet for my 2 &1/2 year old and ...             5.0   

                                    reviews.text_esp  
0  Esta es una tableta muy agradable para mi novi...  
1  Me encanta esta tableta. Fácil de usar. Y el p...  
2  Precio asequible calidad increíble Me encanta ...  
3

# Embeddings

In [3]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from multiprocessing import Pool
import math
import spacy
from num2words import num2words

# Cargar modelo de Spacy para el preprocesamiento
nlp = spacy.load('es_core_news_sm')

# Definir stop words adicionales
stop_words_adicionales = {"\u00a1", "-", "\u2014", "http", "<", ">"}
for palabra in stop_words_adicionales:
    nlp.Defaults.stop_words.add(palabra)

# Función para procesar texto

def procesar_texto(texto):
    oraciones = texto.split('\n')
    total_oraciones = len(oraciones)
    oraciones_tokenizadas = []

    for oracion in oraciones:
        tokens = []
        doc = nlp(oracion)
        for token in doc:
            if token.is_digit:
                try:
                    numero = int(token.text)
                    palabra_letras = num2words(numero, lang='es')
                    tokens.append(palabra_letras)
                except ValueError:
                    pass
            else:
                lemma = token.text.lower()
                if lemma and lemma not in stop_words_adicionales:
                    tokens.append(lemma)
        oraciones_tokenizadas.append(" ".join(tokens))

    return total_oraciones, oraciones_tokenizadas

# Paralelizar el preprocesamiento del texto
def procesar_textos_en_paralelo(data_slice):
    resultados = [procesar_texto(texto) for texto in data_slice['reviews.text_esp']]
    total_oraciones, oraciones_tokenizadas = zip(*resultados)
    return total_oraciones, oraciones_tokenizadas

# Inicializar el modelo BERT
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", ignore_mismatched_sizes=True)

# Función para obtener el embeddin
def obtener_embedding_oracion(oracion):
    inputs = tokenizer(oracion, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Función para obtener el embedding promedio de una reseña completa
def obtener_embedding_resena(oraciones_tokenizadas):
    embeddings_oraciones = [obtener_embedding_oracion(oracion) for oracion in oraciones_tokenizadas]
    return np.mean(embeddings_oraciones, axis=0) if embeddings_oraciones else np.zeros(model.config.hidden_size)

# Paralelizar la generación de embeddings
def procesar_rango(data_slice):
    embeddings = [obtener_embedding_resena(oraciones) for oraciones in data_slice['oraciones_tokenizadas']]
    return embeddings

# Función para dividir los datos en subconjuntos para procesamiento paralelo
def dividir_datos(data, n_cores):
    longitud = len(data)
    p_mas = len(data) % n_cores
    numero_datos = math.floor(longitud / n_cores)
    limites = [(i * numero_datos + min(i, p_mas), (i + 1) * numero_datos + min(i + 1, p_mas)) for i in range(n_cores)]
    return limites

if __name__ == '__main__':
    # Cargar datos (asumiendo que ya están en un DataFrame llamado 'data')
    # data = pd.read_csv('data.csv')

    n_cores = 7

    # Dividir datos para preprocesamiento
    limites = dividir_datos(data, n_cores)
    data_slices = [data.iloc[lim_inf:lim_sup] for lim_inf, lim_sup in limites]

    # Preprocesar textos en paralelo
    with Pool(n_cores) as pool:
        resultados_preprocesamiento = pool.map(procesar_textos_en_paralelo, data_slices)

    # Consolidar resultados del preprocesamiento
    total_oraciones = []
    oraciones_tokenizadas = []
    for total, tokenizadas in resultados_preprocesamiento:
        total_oraciones.extend(total)
        oraciones_tokenizadas.extend(tokenizadas)

    data['total_oraciones'] = total_oraciones
    data['oraciones_tokenizadas'] = oraciones_tokenizadas

    # Dividir datos para generación de embeddings
    limites = dividir_datos(data, n_cores)
    data_slices = [data.iloc[lim_inf:lim_sup] for lim_inf, lim_sup in limites]

    # Generar embeddings en paralelo
    with Pool(n_cores) as pool:
        resultados_embeddings = pool.map(procesar_rango, data_slices)

    # Consolidar resultados de embeddings
    embeddings = [embedding for resultado in resultados_embeddings for embedding in resultado]
    data['embedding_resena'] = embeddings

    # Convertir embeddings a DataFrame y guardar con etiquetas
    embeddings_df = pd.DataFrame(data['embedding_resena'].to_list())
    embeddings_df['sentimiento'] = data['sentimiento'].values
    embeddings_df.to_csv('embeddings_con_etiqueta.csv', index=False)

    print("Embeddings por reseña con etiquetas guardados en 'embeddings_con_etiqueta.csv'.")


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


: 

: 

# Paralelizacion de algoritmos

In [ ]:
import numpy as np
from multiprocessing import Pool
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import time


# utiulizar los mejores hiperparametros dados por grid search previamnet
def entrenar_modelo(tipo_modelo, x_train, y_train, x_test, y_test):
    if tipo_modelo == 'SVM':
        modelo = SVC(C=10, kernel='rbf', gamma =0.01)
    elif tipo_modelo == 'NB':
        modelo = GaussianNB()
    elif tipo_modelo == 'RN':
        modelo = MLPClassifier(hidden_layer_sizes=(50,),activation='relu', solver='adam',max_iter=300)
    else:
        raise ValueError("Tipo de modelo desconocido")
    
    modelo.fit(x_train, y_train)
    y_pred = modelo.predict(x_test)

  
    accuracy = accuracy_score(y_test, y_pred)
    return tipo_modelo, accuracy


if __name__ == '__main__':

    archivo_csv = 'embeddings_con_etiqueta__.csv'
    datos = pd.read_csv(archivo_csv)

    # Mapeo de etiquetas a valores binarios
    datos['sentimiento_binario'] = datos['sentimiento'].map({'NEGATIVO': 0, 'POSITIVO': 1})


    X = datos.iloc[:, :-1].select_dtypes(include=['float', 'int']).values  # Convertir a matriz NumPy
    y = datos['sentimiento_binario'].values  # Convertir a matriz NumPy

 
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Asegurar que los datos no sean solo de lectura
    x_train, x_test = np.copy(x_train), np.copy(x_test)
    y_train, y_test = np.copy(y_train), np.copy(y_test)


    modelos = ['SVM', 'NB', 'RN']

  
    inicio = time.time()

    # Entrenamiento en paralelo
    with Pool(processes=len(modelos)) as pool:
        resultados = pool.starmap(entrenar_modelo, [(modelo, x_train, y_train, x_test, y_test) for modelo in modelos])

    # Mostramos resultados
    for modelo, accuracy in resultados:
        print(f"Modelo: {modelo}, Precisión: {accuracy:.2f}")

    print(f"Tiempo total: {time.time() - inicio:.2f} segundos")

In [ ]:
import numpy as np
from multiprocessing import Pool
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import time

# Utilizar los mejores hiperparámetros dados por grid search previamente
def entrenar_modelo(tipo_modelo, x_train, y_train, x_test, y_test):
    if tipo_modelo == 'SVM':
        modelo = SVC(C=10, kernel='rbf', gamma=0.01)
    elif tipo_modelo == 'NB':
        modelo = GaussianNB()
    elif tipo_modelo == 'RN':
        modelo = MLPClassifier(hidden_layer_sizes=(50,), activation='relu', solver='adam', max_iter=300)
    elif tipo_modelo == 'KMeans':
        # Definir el número de clusters igual a 2 (binario)
        kmeans = KMeans(n_clusters=2, random_state=42)
        kmeans.fit(x_train)

        # Predecir los clusters para los datos de prueba
        clusters_train = kmeans.predict(x_train)
        clusters_test = kmeans.predict(x_test)

        # Evaluar ambas configuraciones de asignación de etiquetas
        accuracy_1 = calcular_precision(clusters_test, y_test, cluster_a=1, cluster_b=0)
        accuracy_2 = calcular_precision(clusters_test, y_test, cluster_a=0, cluster_b=1)

        # Escoger la mejor configuración
        accuracy = max(accuracy_1, accuracy_2)
        return tipo_modelo, accuracy
    else:
        raise ValueError("Tipo de modelo desconocido")

    modelo.fit(x_train, y_train)
    y_pred = modelo.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    return tipo_modelo, accuracy

def calcular_precision(clusters, y_true, cluster_a, cluster_b):
    # Asignar etiquetas a los clusters
    etiquetas_asignadas = np.where(clusters == cluster_a, 1, 0)
    etiquetas_asignadas = np.where(clusters == cluster_b, 0, etiquetas_asignadas)

    # Calcular precisión comparando con las etiquetas reales
    return accuracy_score(y_true, etiquetas_asignadas)

if __name__ == '__main__':
    archivo_csv = 'embeddings_con_etiqueta__.csv'
    datos = pd.read_csv(archivo_csv)

    # Mapeo de etiquetas a valores binarios
    datos['sentimiento_binario'] = datos['sentimiento'].map({'NEGATIVO': 0, 'POSITIVO': 1})

    X = datos.iloc[:, :-1].select_dtypes(include=['float', 'int']).values  # Convertir a matriz NumPy
    y = datos['sentimiento_binario'].values  # Convertir a matriz NumPy

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Asegurar que los datos no sean solo de lectura
    x_train, x_test = np.copy(x_train), np.copy(x_test)
    y_train, y_test = np.copy(y_train), np.copy(y_test)

    # Agregar KMeans a la lista de modelos
    modelos = ['SVM', 'NB', 'RN', 'KMeans']

    inicio = time.time()

    # Entrenamiento en paralelo
    with Pool(processes=len(modelos)) as pool:
        resultados = pool.starmap(entrenar_modelo, [(modelo, x_train, y_train, x_test, y_test) for modelo in modelos])

    # Mostramos resultados
    for modelo, accuracy in resultados:
        print(f"Modelo: {modelo}, Precisi\u00f3n: {accuracy:.2f}")

    print(f"Tiempo total: {time.time() - inicio:.2f} segundos")
